# Network analysis on segment level

Split up the segments into groups of segment type and solar exposure class. Use the resulting files to find critical and favorable segments in the road network.

In [1]:
import warnings
import pandas as pd
import geopandas as gpd
from pathlib import Path

# Ignore FutureWarnings from geopandas
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', message='.*initial implementation of Parquet.*')
# Ignore SettingWithCopyWarning from (geo-)pandas
pd.options.mode.chained_assignment = None  # default='warn'


# Config
tod_list = ['10am', '1pm', '4pm', '7pm']

day = 170
sensitivity_factor = 1.0

in_dir = Path(f'../../export/{day}/{sensitivity_factor}/exportdata/aggregation')
out_dir = in_dir.parent.parent / 'analysis'
out_dir.mkdir(exist_ok=True)
out_segment_level = out_dir / 'segment_level'
out_segment_level.mkdir(exist_ok=True)

default_type = 'shortest'
count_threshold = 0.01

high_sol_expo = 90
low_sol_expo = 50

crs = 'EPSG:4326'

In [2]:
# Helper functions
def subfolder(out_dir, time_of_day):
    """Returns the respective folder path and creates subfolders in out_dir with timestamps if not existent already"""
    out_folder = out_dir / f'{time_of_day}'
    out_folder.mkdir(exist_ok=True)
    return out_folder

In [3]:
# Filters to extract segments ranks 
class EqualSegmentRanking:
    """Filters for equal segment data sets"""

    def __init__(self, data, time_of_day):
        self.data = data.to_crs(crs)
        self.time_of_day = time_of_day

    @property
    def cool_corridors(self):
        filtered = self.data[self.data[f'ranking_{self.time_of_day}'] == f'equal, solar exposure < {low_sol_expo} %']
        
        return filtered

    @property
    def hse_unavoidables(self):
        filtered = self.data[self.data[f'ranking_{self.time_of_day}'] == f'equal, {high_sol_expo} % >= solar exposure']
        
        return filtered
    
    @property
    def mse_unavoidables(self):
        filtered = self.data[self.data[f'ranking_{self.time_of_day}'] == f'equal, {low_sol_expo} % <= solar exposure < {high_sol_expo} %']
        
        return filtered
    
class DetourSegmentRanking:
    """Filters for detour segment data sets"""

    def __init__(self, data, time_of_day):
        self.data = data.to_crs(crs)
        self.time_of_day = time_of_day

    @property
    def cool_detours(self):
        filtered = self.data[self.data[f'ranking_{self.time_of_day}'] == f'detour, solar exposure < {low_sol_expo} %']
        
        return filtered

    @property
    def hse_detours(self):
        filtered = self.data[self.data[f'ranking_{self.time_of_day}'] == f'detour, {high_sol_expo} % >= solar exposure']
            
        return filtered
    
    @property
    def mse_detours(self):
        filtered = self.data[self.data[f'ranking_{self.time_of_day}'] == f'detour, {low_sol_expo} % <= solar exposure < {high_sol_expo} %']
            
        return filtered
    
class DefaultSegmentRanking:
    """Filters the default segment data sets"""

    def __init__(self, data, equal_segments, time_of_day):
        self.time_of_day = time_of_day
        # Avoided default segments
        self.difference = gpd.overlay(data, equal_segments, how='difference')

    @property
    def lse_defaults(self):
        filtered = self.difference[self.difference[f'ranking_{self.time_of_day}'] == f'{default_type}, solar exposure < {low_sol_expo} %']
        
        return filtered

    @property
    def hse_defaults(self):
        filtered = self.difference[self.difference[f'ranking_{self.time_of_day}'] == f'{default_type}, {high_sol_expo} % >= solar exposure']
        
        return filtered
    
    @property
    def mse_defaults(self):
        filtered = self.difference[self.difference[f'ranking_{self.time_of_day}'] == f'{default_type}, {low_sol_expo} % <= solar exposure < {high_sol_expo} %']
        
        return filtered

In [4]:
# Read data of default routes
default_file = gpd.read_feather(in_dir / f'counts_{default_type}_alltimes.feather')
default_file.to_crs(crs, inplace=True)

In [ ]:
# Read and split up segment data of the optimized routes
for time_of_day in tod_list:
    print(f'Processing {time_of_day}...')
    print(f'...equals...')
    equal_segments = EqualSegmentRanking(gpd.read_feather(in_dir / f'counts_{time_of_day}_equal.feather'), time_of_day)

    # Equal + low solar exposure
    lse_equals = equal_segments.cool_corridors
    lse_equals.to_feather(subfolder(out_segment_level, time_of_day) / f'{time_of_day}_cool_corridors.feather')

    # Equal + high solar exposure
    hse_unavoidables = equal_segments.hse_unavoidables
    hse_unavoidables.to_feather(subfolder(out_segment_level, time_of_day) / f'{time_of_day}_hse_unavoidables.feather')
    
    # Equal + medium solar exposure
    mse_unavoidables = equal_segments.mse_unavoidables
    mse_unavoidables.to_feather(subfolder(out_segment_level, time_of_day) / f'{time_of_day}_mse_unavoidables.feather')

    print(f'...detours...')
    detour_segments = DetourSegmentRanking(gpd.read_feather(in_dir / f'counts_{time_of_day}_detour.feather'), time_of_day)

    # Detour + low solar exposure
    lse_detours = detour_segments.cool_detours
    lse_detours.to_feather(subfolder(out_segment_level, time_of_day) / f'{time_of_day}_cool_detours.feather')

    # Detour + high solar exposure
    hse_detours = detour_segments.hse_detours
    hse_detours.to_feather(subfolder(out_segment_level, time_of_day) / f'{time_of_day}_hse_detours.feather')

    # Detour + medium solar exposure
    mse_detours = detour_segments.mse_detours
    mse_detours.to_feather(subfolder(out_segment_level, time_of_day) / f'{time_of_day}_mse_detours.feather')
    
    print(f'...avoided segments...')
    default_segments = DefaultSegmentRanking(default_file, equal_segments.data, time_of_day)

    # Avoided + low solar exposure
    lse_avoided = default_segments.lse_defaults
    lse_avoided.to_feather(subfolder(out_segment_level, time_of_day) / f'{time_of_day}_lse_avoided.feather')
    # Avoided + high solar exposure
    hse_avoided = default_segments.hse_defaults
    hse_avoided.to_feather(subfolder(out_segment_level, time_of_day) / f'{time_of_day}_hse_avoided.feather')
    # Avoided + medium solar exposure
    mse_avoided = default_segments.mse_defaults
    mse_avoided.to_feather(subfolder(out_segment_level, time_of_day) / f'{time_of_day}_mse_avoided.feather')

    avoided = pd.concat([lse_avoided, hse_avoided, mse_avoided], ignore_index=True)
    avoided.to_feather(subfolder(out_segment_level, time_of_day) / f'{time_of_day}_avoided.feather')

print('Done!')

## High-Solar-Exposure (HSE) and Low-Solar-Expsoure (LSE) analysis

Find segments that have high or low solar exposure, regardless if they are shortest or detour segments. This helps in understanding the road network towards critical (HSE) or favorable (LSE) segments.

In [ ]:
# Functions for handling multi-time segment data
def merge_segments(dict):
    """Finds segments of similar solar exposure class"""
    segdata = None

    for gdf in dict.values():
        if segdata is None:
            segdata = gdf
        else:
            segdata = gpd.overlay(segdata, gdf, how='intersection')
            segdata = segdata.loc[:, ~segdata.columns.str.endswith('_1')]
            segdata = segdata.loc[:, ~segdata.columns.str.endswith('_2')]

    return segdata.reset_index(drop=True)

def merge_rest(dict):
    """Merges the remaining segments"""
    return pd.concat(dict.values())

### HSE segments

In [6]:
hse_dict = {}
mse_dict = {}
rest_dict = {}

for tod in tod_list[1:3]:  # slice here to select times of day
    hse_unavoidables = gpd.read_feather(out_segment_level / tod / f'{tod}_hse_unavoidables.feather')
    hse_detours = gpd.read_feather(out_segment_level / tod / f'{tod}_hse_detours.feather')
    hse_dict[f'{tod}_hse'] = hse_unavoidables.append(hse_detours)

    mse_unavoidables = gpd.read_feather(out_segment_level / tod / f'{tod}_mse_unavoidables.feather')
    mse_detours = gpd.read_feather(out_segment_level / tod / f'{tod}_mse_detours.feather')    
    lse_equals = gpd.read_feather(out_segment_level / tod / f'{tod}_cool_corridors.feather')
    lse_detours = gpd.read_feather(out_segment_level / tod / f'{tod}_cool_detours.feather')
    avoided = gpd.read_feather(out_segment_level / tod / f'{tod}_avoided.feather')
    rest_dict[f'{tod}_rest'] = avoided.append(lse_equals).append(lse_detours).append(mse_unavoidables).append(mse_detours)

In [ ]:
hse = merge_segments(hse_dict)
rest = merge_segments(rest_dict)

hse_critical = hse.overlay(rest, how='difference')

In [10]:
# hse_critical.to_feather(out_seg_filter_rank / 'hse_critical.feather')
hse_critical.to_feather(out_segment_level / 'hse_critical_1-4.feather')

### LSE segments

In [11]:
lse_dict = {}
rest_dict = {}

for tod in tod_list[1:3]:  # slice here to select times of day
    lse_equals = gpd.read_feather(out_segment_level / tod / f'{tod}_cool_corridors.feather')
    lse_detours = gpd.read_feather(out_segment_level / tod / f'{tod}_cool_detours.feather')
    lse_dict[f'{tod}'] = lse_equals.append(lse_detours)

    avoided = gpd.read_feather(out_segment_level / tod / f'{tod}_avoided.feather')
    hse_unavoidables = gpd.read_feather(out_segment_level / tod / f'{tod}_hse_unavoidables.feather')
    hse_detours = gpd.read_feather(out_segment_level / tod / f'{tod}_hse_detours.feather')
    mse_unavoidables = gpd.read_feather(out_segment_level / tod / f'{tod}_mse_unavoidables.feather')
    mse_detours = gpd.read_feather(out_segment_level / tod / f'{tod}_mse_detours.feather')   
    rest_dict[f'{tod}'] = avoided.append(hse_unavoidables).append(hse_detours).append(mse_unavoidables).append(mse_detours)

In [ ]:
lse = merge_segments(lse_dict)
rest = merge_segments(rest_dict)

lse_favorable = hse.overlay(rest, how='difference')

In [15]:
# lse_favorable.to_feather(out_seg_filter_rank / 'lse_favorable.feather')
lse_favorable.to_feather(out_segment_level / 'lse_favorable_1-4.feather')